In [1]:
import numpy as np
import pandas as pd

import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, SpatialDropout1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from tensorflow.keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping  

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import matplotlib.pyplot as plt

In [2]:
!pip install xlrd==1.2.0

In [3]:
df = pd.read_excel('отзывы за лето.xls')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [4]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [5]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [6]:
df_train.head()

,Rating,Content,Date
0,5,супер,2017-07-29
1,5,отличный приложение удобно управлять счёт,2017-06-25
2,5,около,2017-08-03
3,5,0 минус,2017-08-02
4,5,приятный положение,2017-07-31


In [7]:
text_corpus_train = df_train['Content'].values
text_corpus_test = df_test['Content'].values

In [8]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False,
                      split = ' ') 

tokenizer.fit_on_texts(text_corpus_train)

In [9]:
sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

In [10]:
word_count = len(tokenizer.index_word) + 1  
training_length = max([len(i.split()) for i in text_corpus_train]) 

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [11]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5], dtype=int64)

In [12]:
num_classes = 5

y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)

y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [13]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count, 
              output_dim=128, 
              input_length=training_length)) 
model.add(Conv1D(128, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
26/26 [==============================] - 3s 82ms/step - loss: 1.0782 - accuracy: 0.6947 - val_loss: 0.8364 - val_accuracy: 0.7049
Epoch 2/20
26/26 [==============================] - 2s 78ms/step - loss: 0.7463 - accuracy: 0.7239 - val_loss: 0.7261 - val_accuracy: 0.7381


In [15]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 0s 17ms/step - loss: 0.7195 - accuracy: 0.7501


Test loss: 0.7194799184799194
Test accuracy: 0.7500806450843811


## RNN

In [16]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64)) 
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [17]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
26/26 [==============================] - 3s 88ms/step - loss: 1.1468 - accuracy: 0.6302 - val_loss: 0.8699 - val_accuracy: 0.7097
Epoch 2/20
26/26 [==============================] - 2s 76ms/step - loss: 0.8633 - accuracy: 0.7190 - val_loss: 0.8020 - val_accuracy: 0.7298


In [18]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 0s 25ms/step - loss: 0.8158 - accuracy: 0.7330


Test loss: 0.8158354759216309
Test accuracy: 0.7329784035682678


In [19]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))  
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
26/26 [==============================] - 7s 218ms/step - loss: 1.2866 - accuracy: 0.6708 - val_loss: 0.9568 - val_accuracy: 0.7049
Epoch 2/20
26/26 [==============================] - 6s 230ms/step - loss: 0.9593 - accuracy: 0.7048 - val_loss: 0.8326 - val_accuracy: 0.7077


In [21]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 61ms/step - loss: 0.8257 - accuracy: 0.7109


Test loss: 0.8256543874740601
Test accuracy: 0.7108744978904724


In [22]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64,recurrent_dropout=0.2))  
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
26/26 [==============================] - 7s 223ms/step - loss: 1.3111 - accuracy: 0.6886 - val_loss: 0.9271 - val_accuracy: 0.7049
Epoch 2/20
26/26 [==============================] - 5s 210ms/step - loss: 0.8751 - accuracy: 0.7049 - val_loss: 0.7782 - val_accuracy: 0.7049


In [24]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 57ms/step - loss: 0.7743 - accuracy: 0.7086


Test loss: 0.7742603421211243
Test accuracy: 0.7086156606674194


## CNN -> RNN

In [25]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64,recurrent_dropout=0.2)) 
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [26]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
26/26 [==============================] - 9s 291ms/step - loss: 1.2459 - accuracy: 0.6737 - val_loss: 0.8811 - val_accuracy: 0.7049
Epoch 2/20
26/26 [==============================] - 7s 280ms/step - loss: 0.8607 - accuracy: 0.7260 - val_loss: 0.7610 - val_accuracy: 0.7353


In [27]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 73ms/step - loss: 0.7597 - accuracy: 0.7430


Test loss: 0.7597280740737915
Test accuracy: 0.7429816126823425


## RNN -> CNN

In [28]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64,recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(64, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [29]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
26/26 [==============================] - 10s 305ms/step - loss: 1.2496 - accuracy: 0.6718 - val_loss: 1.0206 - val_accuracy: 0.7049
Epoch 2/20
26/26 [==============================] - 8s 289ms/step - loss: 1.0624 - accuracy: 0.6979 - val_loss: 0.9844 - val_accuracy: 0.7049


In [31]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 63ms/step - loss: 0.9750 - accuracy: 0.7086


Test loss: 0.9750204682350159
Test accuracy: 0.7086156606674194
